# LCS Code

Let's see how well the code matches the approach we sketched out...

In [1]:
from numpy import *

def findLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1]):  # test for a match ("diagonal street")
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

v = "ATGTTAT"
w = "ATCGTAC"
s, b = findLCS(v,w)
for i in range(len(s)):
    print("%10s %-20s    %12s %-20s" % ('' if i else 'score =', str(s[i]), '' if i else 'backtrack =', str(b[i])))

   score = [0 0 0 0 0 0 0 0]        backtrack = [0 0 0 0 0 0 0 0]   
           [0 1 1 1 1 1 1 1]                    [0 3 2 2 2 2 3 2]   
           [0 1 2 2 2 2 2 2]                    [0 1 3 2 2 3 2 2]   
           [0 1 2 2 3 3 3 3]                    [0 1 1 2 3 2 2 2]   
           [0 1 2 2 3 4 4 4]                    [0 1 3 2 1 3 2 2]   
           [0 1 2 2 3 4 4 4]                    [0 1 3 2 1 3 2 2]   
           [0 1 2 2 3 4 5 5]                    [0 3 1 2 1 1 3 2]   
           [0 1 2 2 3 4 5 5]                    [0 1 3 2 1 3 1 2]   


# Code to extract an answer

A simple recursive LCS() routine to return along the path of arrows that led to our best score.

In [2]:
def LCS(b,v,i,j):
    if ((i == 0) and (j == 0)):
        return ''
    elif (b[i,j] == 3):
        return LCS(b,v,i-1,j-1) + v[i-1]
    elif (b[i,j] == 2):
        return LCS(b,v,i,j-1)
    else:
        return LCS(b,v,i-1,j)
        
print(LCS(b,v,b.shape[0]-1,b.shape[1]-1))

ATGTA


# An alignment of v and w

In [7]:
def Alignment(b,v,w,i,j):
    if ((i == 0) and (j == 0)):
        return ['','']
    if (b[i,j] == 3):
        result = Alignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = Alignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = Alignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result

align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)
print("v =", align[0])
print("w =", align[1])

('v =', 'AT_GTTAT_')
('w =', 'ATCG_TA_C')


In [8]:
v = "SHOULDLOOKLIKETHIS"
w = "SHOUTLOOKSLIKEHIS"
s, b = findLCS(v,w)
print s
print b
align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)
print("v =", align[0])
print("w =", align[1])

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1]
 [ 0  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2]
 [ 0  1  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3]
 [ 0  1  2  3  4  4  4  4  4  4  4  4  4  4  4  4  4  4]
 [ 0  1  2  3  4  4  5  5  5  5  5  5  5  5  5  5  5  5]
 [ 0  1  2  3  4  4  5  5  5  5  5  5  5  5  5  5  5  5]
 [ 0  1  2  3  4  4  5  5  5  5  5  6  6  6  6  6  6  6]
 [ 0  1  2  3  4  4  5  6  6  6  6  6  6  6  6  6  6  6]
 [ 0  1  2  3  4  4  5  6  7  7  7  7  7  7  7  7  7  7]
 [ 0  1  2  3  4  4  5  6  7  8  8  8  8  8  8  8  8  8]
 [ 0  1  2  3  4  4  5  6  7  8  8  9  9  9  9  9  9  9]
 [ 0  1  2  3  4  4  5  6  7  8  8  9 10 10 10 10 10 10]
 [ 0  1  2  3  4  4  5  6  7  8  8  9 10 11 11 11 11 11]
 [ 0  1  2  3  4  4  5  6  7  8  8  9 10 11 12 12 12 12]
 [ 0  1  2  3  4  5  5  6  7  8  8  9 10 11 12 12 12 12]
 [ 0  1  2  3  4  5  5  6  7  8  8  9 10 11 12 13 13 13]
 [ 0  1  2  3  4  5  5  6  7  8

# Global Alignment using a scoring matrix

In [9]:
import numpy

def GlobalAlign(v, w, scorematrix, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(0,len(v)+1):
        for j in range(0,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = s[i-1,j] + indel
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = s[i,j-1] + indel
                b[i,j] = 2
                continue
            score = s[i-1,j-1] + scorematrix[v[i-1],w[j-1]]
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            else:
                b[i,j] = 3
    return (s, b)

match = {('A','A'):  1, ('A','C'): -2, ('A','G'): -1, ('A','T'): -2,
         ('C','A'): -2, ('C','C'):  1, ('C','G'): -2, ('C','T'): -1,
         ('G','A'): -1, ('G','C'): -2, ('G','G'):  1, ('G','T'): -2,
         ('T','A'): -2, ('T','C'): -1, ('T','G'): -2, ('T','T'):  1}

v = "TTCCGAGCGTTA"
w = "TTCCGGTTA"

s, b = GlobalAlign(v,w,match,-3)
print s
print("Best score =", s[-1,-1])
print b
align = Alignment(b,v,w,b.shape[0]-1,b.shape[1]-1)
print("v =", align[0])
print("w =", align[1])

[[  0  -3  -6  -9 -12 -15 -18 -21 -24 -27]
 [ -3   1  -2  -5  -8 -11 -14 -17 -20 -23]
 [ -6  -2   2  -1  -4  -7 -10 -13 -16 -19]
 [ -9  -5  -1   3   0  -3  -6  -9 -12 -15]
 [-12  -8  -4   0   4   1  -2  -5  -8 -11]
 [-15 -11  -7  -3   1   5   2  -1  -4  -7]
 [-18 -14 -10  -6  -2   2   4   1  -2  -3]
 [-21 -17 -13  -9  -5  -1   3   2  -1  -3]
 [-24 -20 -16 -12  -8  -4   0   2   1  -2]
 [-27 -23 -19 -15 -11  -7  -3  -1   0   0]
 [-30 -26 -22 -18 -14 -10  -6  -2   0  -2]
 [-33 -29 -25 -21 -17 -13  -9  -5  -1  -2]
 [-36 -32 -28 -24 -20 -16 -12  -8  -4   0]]
('Best score =', 0)
[[0 2 2 2 2 2 2 2 2 2]
 [1 3 2 2 2 2 2 2 2 2]
 [1 1 3 2 2 2 2 2 2 2]
 [1 1 1 3 2 2 2 2 2 2]
 [1 1 1 1 3 2 2 2 2 2]
 [1 1 1 1 1 3 2 2 2 2]
 [1 1 1 1 1 1 3 2 2 3]
 [1 1 1 1 1 1 3 3 2 3]
 [1 1 1 1 1 1 1 3 3 2]
 [1 1 1 1 1 1 1 1 3 3]
 [1 1 1 1 1 1 1 3 3 3]
 [1 1 1 1 1 1 1 1 3 3]
 [1 1 1 1 1 1 1 1 1 3]]
('v =', 'TTCCGAGCGTTA')
('w =', 'TTCCG_G__TTA')


In [3]:
from numpy import *

v = "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN"
w = "MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVELGGGLGGLQALALEGPPQKRGIVEQCCTSICSLYQLENYCN"

zeros((len(v)+1,len(w)+1), dtype="int32")

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [22]:
[[0 for i in range(len(v)+1)] for i in range(len(w)+1)]

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [17]:
zeros((len(v)+1,len(w)+1), dtype="int32")

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [32]:
from numpy import *

def findLCS(v, w):
    score = zeros((len(v)+1,len(w)+1), dtype="int32")
    backt = zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            # find best score at each vertex
            if (v[i-1] == w[j-1]):  # test for a match ("diagonal street")
                score[i,j], backt[i,j] = max((score[i-1,j-1]+1,3), (score[i-1,j],1), (score[i,j-1],2))
            else:
                score[i,j], backt[i,j] = max((score[i-1,j],1), (score[i,j-1],2))
    return score, backt

v = "abcde"
w = "ace"
s, b = findLCS(v,w)
for i in range(len(s)):
    print("%10s %-20s    %12s %-20s" % ('' if i else 'score =', str(s[i]), '' if i else 'backtrack =', str(b[i])))

   score = [0 0 0 0]                backtrack = [0 0 0 0]           
           [0 1 1 1]                            [0 3 2 2]           
           [0 1 1 1]                            [0 1 2 2]           
           [0 1 2 2]                            [0 1 3 2]           
           [0 1 2 2]                            [0 1 1 2]           
           [0 1 2 3]                            [0 1 1 3]           


In [50]:
class Solution:
    def longestCommonSubsequence(self, text1: str, text2: str) -> int:
        s, b = self.fill_matrices(text1, text2)
        lcs = self.recoverLCS(b, text1, len(text1)-1, len(text2)-1)
        return len(lcs)
    
    def fill_matrices(self, text1, text2):
        score = [[0 for j in range(len(text2)+1)] for i in range(len(text1)+1)]
        backt = [[0 for j in range(len(text2)+1)] for i in range(len(text1)+1)]
        
        for i in range(1, len(text1)+1):
            for j in range(1, len(text2)+1):
                if (text1[i-1] == text2[j-1]):  # test for a match ("diagonal street")
                    score[i][j], backt[i][j] = max((score[i-1][j-1]+1,3), (score[i-1][j],1), (score[i][j-1],2))
                else:
                    score[i][j], backt[i][j] = max((score[i-1][j],1), (score[i][j-1],2))
        return score, backt
    
    def recoverLCS(self, backt, text1, i, j):
        if i == 0 and j == 0:
            return ''
        if backt[i][j] == 3:
            return self.recoverLCS(backt, text1, i-1, j-1) + text1[i-1]
        elif backt[i][j] == 2:
            return self.recoverLCS(backt, text1, i, j-1)
        else:
            return self.recoverLCS(backt, text1, i-1, j)

In [51]:
s = Solution()
s.longestCommonSubsequence("abcde", "ace")

2